In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

from datetime import datetime

#import os

In [2]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/{}'

# def read_database(file_name):
    #return pd.read_csv(os.path.join(URL_BASE, file_name))

def read_from_database(file_name, parse_dates_list=[]):
    return pd.read_csv(URL_BASE.format(file_name), parse_dates=parse_dates_list)

titlesize = 16
labelsize = 16
legendsize = 16
xticksize = 16
yticksize = xticksize

plt.rcParams['legend.markerscale'] = 1.5     # the relative size of legend markers vs. original
plt.rcParams['legend.handletextpad'] = 0.5
plt.rcParams['legend.labelspacing'] = 0.4    # the vertical space between the legend entries in fraction of fontsize
plt.rcParams['legend.borderpad'] = 0.5       # border whitespace in fontsize units
plt.rcParams['font.size'] = 12
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['axes.labelsize'] = labelsize
plt.rcParams['axes.titlesize'] = titlesize
plt.rcParams['figure.figsize'] = (10, 6)

plt.rc('xtick', labelsize=xticksize)
plt.rc('ytick', labelsize=yticksize)
plt.rc('legend', fontsize=legendsize)

### Задача 1. Оценка вероятностей

40% заказов имеют стоимость больше 1000 рублей каждый.
Допустим, новый заказ стоит больше 1000 рублей с вероятностью 0.4.
Оцените вероятность того, что из 100 новых заказов 50 или более стоят дороже 1000 рублей.

Ответ округлите с точность до 3 знака после точки.

(В задаче используется биномиальное распределение:
советуем изучить реализации этого распределения в пакетах: `numpy`, `scipy.stats`.)

In [3]:
n = 100
k = 50
p = 0.4 #Probability of success (order costing more than 1000 RUB).

In [4]:
"""
Разбор: Вариант 3.

Вероятность получить 50 или больше заказов стоимостью больше 1000 рублей
равна единице минус вероятность получить 49 или меньше заказов стоимость больше 1000 рублей.
Последняя вероятность выражается через функцию распределения, которой мы и воспользуемся.
"""

p_49_or_less = stats.binom.cdf(k-1, n, p) #0-49
p_50_or_more = 1 - p_49_or_less #50-100

st_res = round(p_50_or_more, 3)

st_res2 = round(stats.binom.sf(k-1, n, p), 3) #Survival function (1-cdf) variant.

st_res, st_res2

(0.027, 0.027)

In [5]:
"""
Разбор: Вариант 1.

Сгенерируем информацию о 100 заказах, каждый из которых с вероятностью 40% стоит больше 1000.
Количество заказов со стоимостью больше 1000 - это биномиальная случайная величина с параметрами n=100, p=0.4.
Повторим это много раз и численно оценим искомую вероятность.
"""

size = 1_000_000

np_res = round((np.random.binomial(n, p, size) >= k).mean(), 3)
np_res2 = round(1.0 - sum(np.random.binomial(n, p, size) < k) / size, 3)

np_res, np_res2

(0.027, 0.027)

In [6]:
"""
Разбор: Вариант 2.

Посчитаем вероятности получить ровно K заказов со стоимостью больше 1000 для K от 50 до 100.
Просуммируем полученные значения.

(Довольно оскорбительно из-за цикла и накапливающейся в нём ошибки округления, но ОК.)
"""

res = 0.0

for i in range(50, 101, 1):
    res += stats.binom.pmf(i, n, p)
    
round(res, 3)

0.027

### Задача 2. Популярные товары

In [7]:
"""
Определите топ-3 товара по суммарной выручке.
Для решения используйте данные из файла 2022-04-01/2022-04-01T12_df_sales_detail.csv.
Описание данных можно посмотреть на платформе А/Б тестирования на вкладке Главная.
"""

#df = pd.read_csv('./data/2022-04-01T12_df_sales_detail.csv')
df_popular = read_from_database('2022-04-01T12_df_sales_detail.csv')
df_popular.head()

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8
3,1000003,cheese pizza,600,2022-02-04 10:02:35,23420a
4,1000003,italian pizza,720,2022-02-04 10:02:35,23420a


In [8]:
df_popular.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416481 entries, 0 to 416480
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   sale_id  416481 non-null  int64 
 1   good     416481 non-null  object
 2   price    416481 non-null  int64 
 3   date     416481 non-null  object
 4   user_id  416481 non-null  object
dtypes: int64(2), object(3)
memory usage: 15.9+ MB


In [9]:
df_popular.describe()

,sale_id,price
count,4.164810e+05,416481.000000
mean,1.101871e+06,590.284815
std,5.882355e+04,256.404911
min,1.000001e+06,60.000000
25%,1.050803e+06,540.000000
50%,1.101900e+06,660.000000
75%,1.152683e+06,780.000000
max,1.203847e+06,840.000000


In [10]:
popular_goods = (
    df_popular
    .groupby('good')
    ['price'].sum()
    .sort_values(ascending=False)
    .head(3)
    #.index.to_list()
)

popular_goods

good
chefs pizza               24558240
double pepperoni pizza    22558380
chicken bbq pizza         17622540
Name: price, dtype: int64

### Задача 3. Время от захода на сайт до покупки

Оцените средний срок, который проходит от захода пользователя на сайт до совершения покупки.
Будем считать, что заход на сайт относится к покупке, если он был совершён не ранее,
чем за два часа до совершения покупки. Другими словами, для каждой покупки нужно посчитать время,
которое проходит от покупки до первого захода на сайт того же пользователя в течение двух часов перед покупкой,
и от полученных значений времени посчитать среднее.

Для решения используйте данные из файлов `2022-04-01T12_df_sales.csv` и `2022-04-01T12_df_web_logs.csv`.

Полученный результат переведите в минуты и округлите до целого значения.

In [11]:
df_sales = read_from_database('2022-04-01T12_df_sales.csv', [1])
df_web_logs = read_from_database('2022-04-01T12_df_web_logs.csv', [2])

# df_sales = df_sales.loc[(df_sales['user_id']=='f25239'), :]
# df_web_logs = df_web_logs.loc[(df_web_logs['user_id']=='f25239'), :]

In [12]:
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [13]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203847 entries, 0 to 203846
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   sale_id      203847 non-null  int64         
 1   date         203847 non-null  datetime64[ns]
 2   count_pizza  203847 non-null  int64         
 3   count_drink  203847 non-null  int64         
 4   price        203847 non-null  int64         
 5   user_id      203847 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 9.3+ MB


In [14]:
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [15]:
df_web_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1964507 entries, 0 to 1964506
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    object        
 1   page       object        
 2   date       datetime64[ns]
 3   load_time  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 60.0+ MB


In [16]:
df_web_logs.isna().sum()

user_id      0
page         0
date         0
load_time    0
dtype: int64

In [17]:
df_web_sales = (
    df_web_logs.loc[:, ['user_id', 'date']]
    .merge(
        df_sales.loc[:, ['user_id', 'date']],
        how='inner',
        on='user_id',
        suffixes=('_web', '_sales')
    )
)

df_web_sales['time_to_purchase_sec'] = (
    (df_web_sales['date_sales']-df_web_sales['date_web'])
    .astype('timedelta64[s]')
)
    
df_web_sales = (
    df_web_sales
    .loc[
        (df_web_sales['time_to_purchase_sec'] <= 2.0 * 60 * 60)
        & (df_web_sales['time_to_purchase_sec'] >= 0.0),
        :
    ]
)

df_web_sales.sort_values(['user_id', 'date_web', 'date_sales']).head()

,user_id,date_web,date_sales,time_to_purchase_sec
4269382,000096,2022-03-04 10:58:01,2022-03-04 11:15:55,1074.0
4269384,000096,2022-03-04 11:00:02,2022-03-04 11:15:55,953.0
4269386,000096,2022-03-04 11:02:40,2022-03-04 11:15:55,795.0
4269388,000096,2022-03-04 11:04:37,2022-03-04 11:15:55,678.0
4269390,000096,2022-03-04 11:12:36,2022-03-04 11:15:55,199.0


In [18]:
df_web_sales = (
    df_web_sales
    .groupby(['user_id', 'date_sales'])
    ['date_web'].min()
    .reset_index()
    .loc[:, ['user_id', 'date_web', 'date_sales']]
)
         
df_web_sales.head()

,user_id,date_web,date_sales
0,000096,2022-03-04 10:58:01,2022-03-04 11:15:55
1,000096,2022-03-22 12:57:54,2022-03-22 13:16:09
2,0000d4,2022-02-28 16:13:47,2022-02-28 16:32:09
3,0000d4,2022-03-27 11:09:14,2022-03-27 11:26:30
4,0000de,2022-02-11 18:41:22,2022-02-11 18:57:15


In [19]:
df_web_sales['time_to_purchase_sec'] = (
    (df_web_sales['date_sales']-df_web_sales['date_web'])
    .astype('timedelta64[s]')
)

In [20]:
df_web_sales['time_to_purchase_sec'].describe()

count    203847.000000
mean        995.312671
std         285.584947
min          53.000000
25%         890.000000
50%        1038.000000
75%        1132.000000
max        7199.000000
Name: time_to_purchase_sec, dtype: float64

In [21]:
mean_ttp = round(df_web_sales['time_to_purchase_sec'].mean() / 60.0)
mean_ttp

17

In [22]:
#Good suggesgted solution:
from datetime import timedelta

# объединяем таблицы с покупками и логами по пользователям
# оставляем только user_id и времена
df = pd.merge(
    df_sales[['user_id', 'date']].rename(columns={'date': 'date_sale'}),
    df_web_logs[['user_id', 'date']].rename(columns={'date': 'date_web'}),
    on='user_id',
    how='inner'
)

# удаляем строки, в которых заход на сайт ранее 2 часов до покупки или после покупки
df = df[
    (df['date_web'] >= df['date_sale'] - timedelta(hours=2))
    & (df['date_web'] < df['date_sale'])
].copy()
# определяем время первого захода на сайт для покупки
df_first_web = df.groupby(['user_id', 'date_sale'])[['date_web']].min().reset_index()
# вычисляем время в секундах
df_first_web['delta'] = (df_first_web['date_sale'] - df_first_web['date_web']).dt.total_seconds()

# считаем среднее время и переводим в минуты
answer = round(df_first_web['delta'].mean() / 60)
print('answer:', answer)

answer: 17


### Задача 5. Функция для фильтрации данных

In [23]:
"""
Для оценки результатов экспериментов нам нужно выбрать данные, полученные во время эксперимента.

Напишите функцию get_data_subset.

Шаблон решения
"""

from datetime import datetime, timedelta
import pandas as pd


def get_data_subset(df, begin_date=None, end_date=None, user_ids=None, columns=None):
    """Возвращает подмножество данных.

    :param df (pd.DataFrame): таблица с данными, обязательные столбцы: 'date', 'user_id'.
    :param begin_date (datetime.datetime | None): дата начала интервала с данными.
        Пример, df[df['date'] >= begin_date].
        Если None, то фильтровать не нужно.
    :param end_date (datetime.datetime | None): дата окончания интервала с данными.
        Пример, df[df['date'] < end_date].
        Если None, то фильтровать не нужно.
    :param user_ids (list[str] | None): список user_id, по которым нужно предоставить данные.
        Пример, df[df['user_id'].isin(user_ids)].
        Если None, то фильтровать по user_id не нужно.
    :param columns (list[str] | None): список названий столбцов, по которым нужно предоставить данные.
        Пример, df[columns].
        Если None, то фильтровать по columns не нужно.

    :return df (pd.DataFrame): датафрейм с подмножеством данных.
    """
    begin_date_, end_date_, user_ids_, columns_ = begin_date, end_date, user_ids, columns
    
    if not begin_date_:
        begin_date_ = df['date'].min()
    if not end_date_:
        end_date_ = df['date'].max() + timedelta(days=1)
    if not user_ids_:
        user_ids_ = df['user_id'].unique()
    if not columns_:
        columns_ = df.columns.to_list()
    
    return (
        df
        .loc[
            (df['date'] >= begin_date_)
            & (df['date'] < end_date_)
            & df['user_id'].isin(user_ids_),
            columns_
        ].copy()
    )

In [24]:
df = pd.DataFrame({
    'date': [datetime(2022, 1, 5), datetime(2022, 1, 7)],
    'user_id': ['1', '2'],
})
new_df = get_data_subset(df, datetime(2022, 1, 6))
# new_df = pd.DataFrame(
#     'date': [datetime(2022, 1, 5)],
#     'user_id': ['1'],
# )
new_df

,date,user_id
1,2022-01-07,2


### Задача 6. Функции вычисления метрик

In [25]:
"""
Напишите функции get_response_time, get_revenue_web и get_revenue_all
для вычисления метрик «revenue (web)», «revenue (all)» и «response time».

Шаблон решения
"""

import pandas as pd
from datetime import datetime


def get_response_time(df_web_logs, begin_date, end_date):
    """Вычисляет значения времени обработки запроса сервером.

    Нужно вернуть значения user_id и load_time из таблицы df_web_logs,
    отфильтрованные по дате.
    Считаем, что запросы обрабатываются независимо, поэтому группировать
    по user_id не нужно.

    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
    столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для
    фильтрации данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    return (
        get_data_subset(
            df=df_web_logs,
            begin_date=begin_date,
            end_date=end_date,
            # user_ids=None,
            columns=['user_id', 'load_time']
        ).rename(columns={'load_time': 'metric'})
    )


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт в указанный период.

    Эти данные нужны для экспериментов на сайте, когда в эксперимент
    попадают только те, кто заходил на сайт во время эксперимента.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный
    период для пользователей, заходивших на сайт в указанный период.
    Если пользователь зашёл на сайт и ничего не купил, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_users = pd.DataFrame(
        get_data_subset(
            df=df_web_logs,
            begin_date=begin_date,
            end_date=end_date,
            # user_ids=None,
            columns=['user_id']
        )['user_id'].unique(),
        columns=['user_id']
    )
    
    df_res = (
        df_users
        .merge(
            get_data_subset(
                df=df_sales, 
                begin_date=begin_date, 
                end_date=end_date,
                # user_ids=None,
                columns=['user_id', 'price']
            ).groupby('user_id')
            ['price'].sum()
            .reset_index(),
            how='left',
            on='user_id'
        ).fillna(0)
        .rename(columns={'price': 'metric'})
        .loc[:, ['user_id', 'metric']]
    )
    
    return df_res


def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт до end_date.

    Эти данные нужны, например, для экспериментов с рассылкой по email,
    когда в эксперимент попадают те, кто когда-либо оставил нам свои данные.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный период
    для пользователей, заходивших на сайт до end_date.
    Если пользователь ничего не купил за указанный период, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_users = pd.DataFrame(
        get_data_subset(
            df=df_web_logs,
            # begin_date=None, 
            end_date=end_date,
            # user_ids=None,
            columns=['user_id']
        )['user_id'].unique(),
        columns=['user_id']
    )
    
    df_res = (
        df_users
        .merge(
            get_data_subset(
                df=df_sales, 
                begin_date=begin_date, 
                end_date=end_date,
                # user_ids=None,
                columns=['user_id', 'price']
            ).groupby('user_id')
            ['price'].sum()
            .reset_index(),
            how='left',
            on='user_id'
        ).fillna(0)
        .rename(columns={'price': 'metric'})
        .loc[:, ['user_id', 'metric']]
    )
    
    return df_res

In [26]:
df_sales = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(11, 14)],
    'price': [1100, 900, 1500],
    'user_id': ['1', '2', '1'],
})

df_web_logs = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(10, 14)],
    'load_time': [80.8, 90.1, 15.8, 19.7],
    'user_id': ['3', '1', '2', '1'],
})

begin_date = datetime(2022, 3, 11, 9)
end_date = datetime(2022, 4, 11, 9)

In [27]:
# response_time = pd.DataFrame({'user_id': ['1', '2', '1'], 'metric': [90.1, 15.8, 19.7],})
response_time = get_response_time(df_web_logs, begin_date, end_date)
response_time

,user_id,metric
1,1,90.1
2,2,15.8
3,1,19.7


In [28]:
# revenue_web = pd.DataFrame({'user_id': ['1', '2'], 'metric': [2600, 900],})
revenue_web = get_revenue_web(df_sales, df_web_logs, begin_date, end_date)
revenue_web

,user_id,metric
0,1,2600
1,2,900


In [29]:
# revenue_all = pd.DataFrame({'user_id': ['1', '2', '3'], 'metric': [2600, 900, 0],})
revenue_all = get_revenue_all(df_sales, df_web_logs, begin_date, end_date)
revenue_all

,user_id,metric
0,3,0.0
1,1,2600.0
2,2,900.0


In [30]:
#Suggested solution isn't bad either.

import pandas as pd


def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):
    if begin_date:
        df = df[df['date'] >= begin_date]
    if end_date:
        df = df[df['date'] < end_date]
    if user_ids:
        df = df[df['user_id'].isin(user_ids)]
    if columns:
        df = df[columns]
    return df.copy()


def get_response_time(df_web_logs, begin_date, end_date):
    return (
        get_data_subset(df_web_logs, begin_date, end_date, None, ['user_id', 'load_time'])
        .rename(columns={'load_time': 'metric'})
        [['user_id', 'metric']]
    )


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    df_users = (
        get_data_subset(df_web_logs, begin_date, end_date, None, ['user_id'])
        .drop_duplicates()
    )
    df = (
        get_data_subset(df_sales, begin_date, end_date, None, ['user_id', 'price'])
        .groupby('user_id')[['price']].sum().reset_index() 
        .rename(columns={'price': 'metric'})
    )
    df = pd.merge(df_users, df, on='user_id', how='left').fillna(0)
    return df[['user_id', 'metric']]


def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    df_users = (
        get_data_subset(df_web_logs, None, end_date, None, ['user_id'])
        .drop_duplicates()
    )
    df = (
        get_data_subset(df_sales, begin_date, end_date, None, ['user_id', 'price'])
        .groupby('user_id')[['price']].sum().reset_index() 
        .rename(columns={'price': 'metric'})
    )
    df = pd.merge(df_users, df, on='user_id', how='left').fillna(0)
    return df[['user_id', 'metric']]
